In [3]:
import time
import sklearn
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import make_scorer,mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.preprocessing import MinMaxScaler,StandardScaler,Normalizer,PolynomialFeatures

import utils
import models

# 读取数据
combi = pd.read_csv('Dataset/combi.csv')

# 数据集划分
# train_data = combi[combi['Time'] < '2021-12-22 08:00:00']
# val_data = combi[(combi['Time'] > '2021-12-22 08:00:00') & (combi['Time'] < '2021-12-22 23:30:00')]
Train_data = combi[combi['Time'] < '2021-12-22 23:30:00']
train_data, val_data = train_test_split(Train_data, test_size=0.2, random_state=42)
pred_data = combi[combi['Time'] >= '2021-12-22 23:30:00']

# 特征选择
features = ['2wind', 'SO2', 'Water_level', 'NOx', 'Water_flow', 'Water_flow_shift', 'Steam_flow']

# 导出分析报告
import pandas_profiling
pfr = pandas_profiling.ProfileReport(combi[features])
pfr.to_file("combi.html")

/mnt/DATA1/wuliubin/miniconda3/envs/metro/lib/python3.9/site-packages/multimethod/__init__.py:312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# 数据标准化
scaler = [PolynomialFeatures(len(features)), MinMaxScaler(), StandardScaler(), Normalizer()]
x_train=scaler[0].fit_transform(train_data[features])
x_val =scaler[0].transform(val_data[features])

In [ ]:
# 各模型初步评估
lr = LinearRegression()
rgcv=RidgeCV()
eltcv=ElasticNetCV()
lasso=LassoCV()
# rf =RandomForestRegressor()  # 这个需要很长的时间
gbdt=GradientBoostingRegressor()
xgb =XGBRegressor()
lgbm = LGBMRegressor()
models =[lr,rgcv,eltcv, lasso, gbdt ,xgb,lgbm]

for model in models:
    model=model.fit(x_train,train_data['Steam_flow'])
    predict_val=model.predict(x_val)
    print(model)
    print('val r2_score :',metrics.r2_score(val_data['Steam_flow'],predict_val))
    print('val mean_squared_error :',metrics.mean_squared_error(val_data['Steam_flow'],predict_val))
    print('**********************************')

In [ ]:
model_lgb=LGBMRegressor(random_state=2021)
params_dic=dict(learning_rate=[0.01, 0.1, 1], n_estimators=[20,50,120,300], 
                num_leaves=[10,30],max_depth=[-1,4,10])
grid_search = GridSearchCV(model_lgb, cv=5,
                param_grid=params_dic,
                scoring='neg_mean_squared_error')
grid_search.fit(x_train,train_data['Steam_flow'])
print(f'最好的参数是:{grid_search.best_params_}')
print(f'最好的分数是:{-grid_search.best_score_}')

# 根据得到的参数进行训练
lgb_final =LGBMRegressor(random_state=2021,learning_rate= 0.1, max_depth= 5,
                        n_estimators=200, num_leaves= 50)
lgb_final.fit(x_train,train_data['Steam_flow'])
val_pred =lgb_final.predict(x_val)
print(f"mean_squared_error:{metrics.mean_squared_error(val_data['Steam_flow'],val_pred)}")

In [ ]:
xgb_re=XGBRegressor(seed=27,learning_rate=0.1, n_estimators=300,silent=0, objective='reg:linear',
                         gamma=0,subsample=0.8,colsample_bytree=0.8,nthread=4,scale_pos_weight=1)
xgb_params ={'n_estimators':[50,100,120],'min_child_weight':list(range(1,4,2)),}
best_model= GridSearchCV(xgb_re,param_grid=xgb_params,refit=True,
                        cv=5,scoring='neg_mean_squared_error')
best_model.fit(x_train,train_data['Steam_flow'])
print('best_parameters:',best_model.best_params_) 
print(f'最好的分数是:{-grid_search.best_score_}')

xgb_final=XGBRegressor(seed=27,learning_rate=0.1, objective='reg:linear',
                gamma=0.2,subsample=0.5,
                colsample_bytree=0.8,nthread=1,scale_pos_weight=1,
                    min_child_weight=0.3, n_estimators=300)
                                                    
xgb_final.fit(x_train,train_data['Steam_flow'])
val_pred =xgb_final.predict(x_val)
print(f"mean_squared_error:{metrics.mean_squared_error(val_data['Steam_flow'],val_pred)}")